# CatBoost(Category Boosting)

Kategorik değişkenlerin sayısı çok olduğunda iyidir.

Kategorik değişkenler ile otomatik olarak mücadele edebilen hızlı başarılı bir diğer GBM türevidir.

- kategorig değişken desteği
- Hızlı ve ölçeklenebilir  GPU desteği
- Daha başarılı tahminler
- Hızlı train ve hızlı tahmin
- Rusyanın ilk açık kaynak kodlu başarılı Ml çalışması

In [1]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale, StandardScaler
from sklearn import model_selection
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error, r2_score, roc_auc_score, roc_curve, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier,MLPRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor
from sklearn import neighbors
from sklearn.svm import SVR

In [6]:
df = pd.read_csv("Hitters.csv")
df = df.dropna()
dms = pd.get_dummies(df[["League","Division","NewLeague"]])
#Veri setindeki kategorik değişkenler için dummy değişkenler oluşturduk
#One hot encoding yöntemi ile.
y = df["Salary"] #Bağımlı değişkenimiz
X_ = df.drop(["Salary","League","Division","NewLeague"],axis=1).astype("float64")
#Veri setimizdeki bağımlı değişken ve kategorik değişkenlerimizin orjinal hallerini uçurduk
X = pd.concat([X_,dms[["League_N","Division_W","NewLeague_N"]]],axis = 1)
#Bu kategorik değişkenlerin dummy versyonlarını ve diğer bağımsız
#değişken değerlerimizi bir arada tutarak bağımsız değişkenlerimizi
#X olarak oluşturmuş olduk
X_train,X_test,y_train,y_test  =train_test_split(X,y,test_size = 0.25,random_state=42)

In [7]:
from catboost import CatBoostRegressor

In [9]:
catb_model = CatBoostRegressor().fit(X_train,y_train)

Learning rate set to 0.029229
0:	learn: 438.1974206	total: 2.34ms	remaining: 2.34s
1:	learn: 432.4168868	total: 4.17ms	remaining: 2.08s
2:	learn: 426.3836690	total: 5.72ms	remaining: 1.9s
3:	learn: 420.2261014	total: 7.02ms	remaining: 1.75s
4:	learn: 414.9976675	total: 8.35ms	remaining: 1.66s
5:	learn: 409.6125323	total: 9.43ms	remaining: 1.56s
6:	learn: 403.9277911	total: 10.7ms	remaining: 1.52s
7:	learn: 398.4395285	total: 11.9ms	remaining: 1.47s
8:	learn: 392.4517081	total: 13ms	remaining: 1.43s
9:	learn: 387.4871123	total: 14ms	remaining: 1.39s
10:	learn: 382.6230510	total: 15.2ms	remaining: 1.37s
11:	learn: 378.1012454	total: 16.3ms	remaining: 1.34s
12:	learn: 372.6002306	total: 17.3ms	remaining: 1.31s
13:	learn: 368.4682192	total: 18.3ms	remaining: 1.29s
14:	learn: 364.0565766	total: 19.4ms	remaining: 1.27s
15:	learn: 359.5683249	total: 20.4ms	remaining: 1.25s
16:	learn: 355.1782794	total: 21.5ms	remaining: 1.24s
17:	learn: 350.4689946	total: 22.5ms	remaining: 1.23s
18:	learn: 34

In [12]:
y_pred = catb_model.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

350.2683163098795

### Model Tuning

In [13]:
catb_params = {"iterations":[200,500,100],
              "learning_rate":[0.01,0.1],
              "depth":[3,6,8]}

In [14]:
catb_model = CatBoostRegressor()

In [15]:
cat_cv_model = GridSearchCV(catb_model,catb_params,cv = 5,n_jobs = -1,verbose = 2).fit(X_train,y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:   42.8s finished


0:	learn: 425.7900818	total: 752us	remaining: 150ms
1:	learn: 404.8723520	total: 1.3ms	remaining: 129ms
2:	learn: 387.4057666	total: 1.82ms	remaining: 120ms
3:	learn: 372.2801584	total: 2.31ms	remaining: 113ms
4:	learn: 358.9204229	total: 2.81ms	remaining: 110ms
5:	learn: 347.0083933	total: 3.33ms	remaining: 108ms
6:	learn: 336.0130818	total: 3.85ms	remaining: 106ms
7:	learn: 324.3923300	total: 4.29ms	remaining: 103ms
8:	learn: 314.8690957	total: 4.77ms	remaining: 101ms
9:	learn: 308.5075563	total: 5.28ms	remaining: 100ms
10:	learn: 298.8587285	total: 5.72ms	remaining: 98.4ms
11:	learn: 294.7655438	total: 6.18ms	remaining: 96.8ms
12:	learn: 288.0697862	total: 6.64ms	remaining: 95.5ms
13:	learn: 282.6697154	total: 7.1ms	remaining: 94.4ms
14:	learn: 277.6121667	total: 7.55ms	remaining: 93.2ms
15:	learn: 273.4383979	total: 8.01ms	remaining: 92.1ms
16:	learn: 269.1556201	total: 8.54ms	remaining: 91.9ms
17:	learn: 264.8098704	total: 9.08ms	remaining: 91.8ms
18:	learn: 261.6700768	total: 9.5

In [17]:
cat_cv_model.best_params_

{'depth': 3, 'iterations': 200, 'learning_rate': 0.1}

In [18]:
catb_Tuned = CatBoostRegressor(depth=3,iterations=200,learning_rate=0.1).fit(X_train,y_train)

0:	learn: 425.7900818	total: 741us	remaining: 148ms
1:	learn: 404.8723520	total: 1.23ms	remaining: 122ms
2:	learn: 387.4057666	total: 1.69ms	remaining: 111ms
3:	learn: 372.2801584	total: 2.23ms	remaining: 109ms
4:	learn: 358.9204229	total: 2.78ms	remaining: 109ms
5:	learn: 347.0083933	total: 3.54ms	remaining: 115ms
6:	learn: 336.0130818	total: 4.12ms	remaining: 114ms
7:	learn: 324.3923300	total: 4.66ms	remaining: 112ms
8:	learn: 314.8690957	total: 5.22ms	remaining: 111ms
9:	learn: 308.5075563	total: 5.74ms	remaining: 109ms
10:	learn: 298.8587285	total: 6.28ms	remaining: 108ms
11:	learn: 294.7655438	total: 6.75ms	remaining: 106ms
12:	learn: 288.0697862	total: 7.2ms	remaining: 104ms
13:	learn: 282.6697154	total: 7.64ms	remaining: 102ms
14:	learn: 277.6121667	total: 8.09ms	remaining: 99.8ms
15:	learn: 273.4383979	total: 8.56ms	remaining: 98.5ms
16:	learn: 269.1556201	total: 9.04ms	remaining: 97.3ms
17:	learn: 264.8098704	total: 9.49ms	remaining: 96ms
18:	learn: 261.6700768	total: 10.1ms	r

In [19]:
y_pred = catb_Tuned.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

344.3125832615482